In [ ]:
import copy
import torch
from torch import nn
from torch import optim
import torchtext
from torchtext import data
from torchtext import datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

# hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=32)

problem 1


In [ ]:
# pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    correct_preds = 0
    total_preds = 0

    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Calculate training accuracy
        preds = torch.argmax(predictions, dim=1)
        correct_preds += (preds == batch.label).sum().item()
        total_preds += len(batch.label)

    accuracy = correct_preds / total_preds
    return epoch_loss / len(iterator), accuracy


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, batch.label)
            epoch_loss += loss.item()

            # Calculate accuracy
            preds = torch.argmax(predictions, dim=1)
            correct_preds += (preds == batch.label).sum().item()
            total_preds += len(batch.label)

    accuracy = correct_preds / total_preds
    return epoch_loss / len(iterator), accuracy


In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, padding_idx):
        super(RNNClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.num_layers = 1
        # 1. Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=padding_idx)

        self.rnn = nn.RNN(self.embedding_dim, self.hidden_dim, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(self.hidden_dim, self.hidden_dim+200)
        self.fc2= nn.Linear(self.hidden_dim+200, self.label_size)


    def zero_state(self, batch_size):
        # implement the function, which returns an initial hidden state.
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        return h0


    def forward(self, text):

        embedded = self.embedding(text)


        batch_size = text.size(0)
        h0 = self.zero_state(batch_size)
        rnn_out, _ = self.rnn(embedded, h0)

        final_hidden_state = rnn_out[:, -1, :]

        x = self.fc1(final_hidden_state)
        logits=self.fc2(x)

        return logits

In [ ]:

model = RNNClassifier(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx)

In [ ]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
N_EPOCHS = 155
train_acc_arr=[]
val_acc_arr=[]
train_lss_arr=[]
test_lss_arr=[]
test_acc_arr=[]
val_lss_arr=[]

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iter, optimizer, criterion)  # Pass device
    val_loss, val_acc = evaluate(model, val_iter, criterion)
    test_loss, test_acc = evaluate(model, test_iter, criterion)
    test_lss_arr.append(test_loss)
    test_acc_arr.append(test_acc)
    train_acc_arr.append(train_acc)
    val_acc_arr.append(val_acc)
    train_lss_arr.append(train_loss)
    val_lss_arr.append(val_loss)
    print("Epoch",epoch)
    print("Train loss",train_loss,"Train acc",train_acc)
    print("Val loss",val_loss,"Val acc",val_acc)
    print("Test loss",test_loss,"Test acc",test_acc)



Epoch 0
Train loss 1.0498058519113376 Train acc 0.41420880149812733
Val loss 1.0540975860186985 Val acc 0.40599455040871935
Test loss 1.034387070792062 Test acc 0.4122171945701357
Epoch 1
Train loss 1.0465956276722168 Train acc 0.4229868913857678
Val loss 1.0387198311941965 Val acc 0.4396003633060854
Test loss 1.0360353725297111 Test acc 0.43619909502262444
Epoch 2
Train loss 1.0458778267942563 Train acc 0.4225187265917603
Val loss 1.0383976476533072 Val acc 0.41326067211625794
Test loss 1.037851448569979 Test acc 0.42217194570135747
Epoch 3
Train loss 1.0464190266998519 Train acc 0.42310393258426965
Val loss 1.0575488005365645 Val acc 0.40236148955495005
Test loss 1.0362409106322696 Test acc 0.41312217194570133
Epoch 4
Train loss 1.045999890186367 Train acc 0.4233380149812734
Val loss 1.0635463782719203 Val acc 0.40236148955495005
Test loss 1.037119379213878 Test acc 0.4113122171945701
Epoch 5
Train loss 1.0452343886264701 Train acc 0.4243913857677903
Val loss 1.0559881619044713 Val a

In [ ]:
import numpy as np
x_xis=np.arange(0,50)

Training with different model for higher degree of accuracy